## Minimizing memory in pandas

In [ ]:
# !pip install pandas-downcast

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import pdcast as pdc
import numpy as np

### Example from PyPI

In [ ]:
# big_int = 1e15
# a = np.linspace(big_int + 1, big_int + 100 , 100, dtype=int)
# a[0]


In [ ]:
big_int = 1e15
data = {
    "integers": np.linspace(1, 100, 100, dtype=int),
    "big_integers": np.linspace(big_int + 1, big_int + 100 , 100, dtype=int),
    "floats": np.linspace(1, 1000, 100).round(2),
    "booleans": np.random.choice([True, False], 100),
    "binaries": np.random.choice([0, 1], 100),
    "categories": np.random.choice(["foo", "bar", "baz"], 100),
}


df = pd.DataFrame(data)
    
# make the df bigger
df = pd.concat([df] * 5_000)

print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
raw_mem = df.memory_usage(deep=True).sum() / 1024**2
raw_mem

In [ ]:
# note that it doensn't handle nans well by default if the other float values in the column can be cast to ints
df_temp = df.copy()
df_temp['integers'] = np.where(df_temp['integers'] <= 3, np.nan, df['integers'])
df_temp['big_integers'] = np.where(df_temp.index <= 3, np.nan, df['big_integers'])
df_temp.head()

In [ ]:
df_downcast_temp = pdc.downcast(df_temp)
df_downcast_temp.info()


In [ ]:
# Downcast DataFrame to minimum viable schema.


df_downcast = pdc.downcast(df)
df_downcast.info()

In [ ]:
# Infer minimum schema for DataFrame.
schema = pdc.infer_schema(df)
schema

In [ ]:
# Coerce DataFrame to schema - required if converting float to Pandas Integer.
df_new = pdc.coerce_df(df, schema)
dcast_mem = df_new.memory_usage(deep=True).sum() / 1024**2
dcast_mem

In [ ]:
# new dataset is what percentage smaller than the original
mem_savings_as_perc = 100 * (raw_mem - dcast_mem) / raw_mem
mem_savings_as_perc

### Flights Data

In [ ]:
%%time
# read in data (flights or parking violations)

my_cols = [
    "Cancelled",
    "CancellationCode",
    "Diverted",
    "LateAircraftDelay"]

flights = pd.read_csv("../data/DelayedFlights.csv")#, usecols=None)
# df = pd.read_csv("parkingviolations-copy-from-files-api.csv", low_memory=False)
print(flights.shape)
flights.head(3)

In [ ]:
flights.info(show_counts=True)

In [ ]:
flights.memory_usage(deep=True).sum() / 1024**2

In [ ]:
# Downcast DataFrame to minimum viable schema.
df_downcast = pdc.downcast(flights.head(10))
df_downcast.info()

# # Infer minimum schema for DataFrame.
# schema = pdc.infer_schema(df)

# # Coerce DataFrame to schema - required if converting float to Pandas Integer.
# df_new = pdc.coerce_df(df, schema)

In [ ]:
# find the problem column
cols_used = []
for c in flights.columns:
    print(c)
    cols_used.append(c)
    df_downcast = pdc.downcast(flights[cols_used])


In [ ]:
df_downcast = pdc.downcast(flights[["CRSElapsedTime"]].head(10))


In [ ]:
temp = flights[["CRSElapsedTime"]].copy().iloc[150_000:150_500,:]
df_downcast = pdc.downcast(temp)
df_downcast.head()

In [ ]:
# Infer minimum schema for DataFrame.
flights_schema = pdc.infer_schema(flights)
flights_schema

# Coerce DataFrame to schema - required if converting float to Pandas Integer.
flights_new = pdc.coerce_df(flights, flights_schema)

In [ ]:
flights_new

In [ ]:
# Create dataframe using dictionary
# data = {'Student ID': [10, 11, 12, 13, 14], 
#         'Age': [23, 22, 24, 22, 25],
#         'Weight': [66, 72, np.inf, 68, -np.inf]}
  
df = flights
  
# display(df)
  
# checking for infinity
print()
print("checking for infinity")
  
ds = df.isin([np.inf, -np.inf])
# print(ds)
  
# printing the count of infinity values
print()
print("printing the count of infinity values")
  
count = np.isinf(df).values.sum()
print("It contains " + str(count) + " infinite values")
  
# counting infinity in a particular column name
c = np.isinf(df['Weight']).values.sum()
print("It contains " + str(c) + " infinite values")
  
# printing column name where infinity is present
print()
print("printing column name where infinity is present")
col_name = df.columns.to_series()[np.isinf(df).any()]
print(col_name)
  
# printing row index with infinity
print()
print("printing row index with infinity ")
  
r = df.index[np.isinf(df).any(1)]
print(r)

### using the downcast library

In [ ]:
!pip install pandas-downcast

In [ ]:
df.columns.tolist()
# keeps = [
#     "Year",
#     "Month",
#     "DayofMonth",
#     "DayOfWeek",
#     "DepTime",
#     "CRSDepTime",
#     "ArrTime",
#     "CRSArrTime",
#     "UniqueCarrier",
#     "FlightNum",
#     "TailNum",
#     "ActualElapsedTime",
#     "CRSElapsedTime",
#     "AirTime",
#     "Origin",
#     "Dest",
#     "Distance",
#     "TaxiIn",
#     "TaxiOut",
#     "Cancelled",
#     "CancellationCode",
#     "Diverted",
#     "LateAircraftDelay"
# ]
# df[keeps].memory_usage(deep=True).sum() / 1024 ** 2

In [ ]:
# df[keeps].info()

In [ ]:
# df = df[keeps]

In [ ]:
# memory usage in megabytes
# df.memory_usage(deep=True).sum() / 1_000_000
df.memory_usage(deep=True).sum() / 1024 ** 2

In [ ]:
df.info(memory_usage='deep')

In [ ]:
for dtype in ['float','int','object']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [ ]:
# ranges of possible values for each integer subtype
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

In [ ]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
# convert integer columns
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')


print(mem_usage(df_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([df_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
# convert float columns
df_float = df.select_dtypes(include=['float'])
converted_float = df_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(df_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([df_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

In [ ]:
# see how total df memory compares after all numeric conversions
optimized_df = df.copy()
optimized_df[converted_int.columns] = converted_int
optimized_df[converted_float.columns] = converted_float

print(mem_usage(df))
print(mem_usage(optimized_df))

#### Object Memory Notes
- object types use a variable amount of memory depending on what is stored within each string

In [ ]:
from sys import getsizeof
s1 = 'working out'
s2 = 'memory usage for'
s3 = 'strings in python is fun!'
s4 = 'strings in python is fun!'
for s in [s1, s2, s3, s4]:
    print(getsizeof(s))

In [ ]:
# look at the obect columns
df_obj = df.select_dtypes(include=['object']).copy()
df_obj.describe()

In [ ]:
# convert to category type as long as the number of unique values is less than 50% of the total values
converted_obj = pd.DataFrame()
for col in df_obj.columns:
    num_unique_values = len(df_obj[col].unique())
    num_total_values = len(df_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = df_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = df_obj[col]

In [ ]:
# compare memory usage of object cols vs category
print(mem_usage(df_obj))
print(mem_usage(converted_obj))
compare_obj = pd.concat([df_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
optimized_df[converted_obj.columns] = converted_obj
print(mem_usage(df))
print(mem_usage(optimized_df))

In [ ]:
optimized_df.info()

In [ ]:
dtypes = optimized_df.dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint
preview = first2pairs = {key:value for key,value in list(column_types.items())}
print(preview)